In [1]:
#!pip install open3d
#!conda install -c open3d-admin open3d

In [4]:
import open3d as o3d
import numpy as np

In [5]:
#pointCloudMatrix = [[0.0, 0, 0],[0, 0, 1],[0, 1, 0],[0, 1, 1],[1, 0, 0],[1, 0, 1],[1, 1, 0],[1, 1, 1]];
pointCloudMatrix = np.array([[0,0,0]])
for x in range(101):
    for y in range(101):
        for z in range(101):
            if (abs(x/100) == 1 or abs(y/100) == 1 or abs(z/100)==1 or abs(x) == 0 or abs(y) == 0 or abs(z)==0):
                pointCloudMatrix = np.concatenate((pointCloudMatrix,[[x,y,z]]),axis=0)
                
           


In [6]:
pointCloudMatrix.shape

(60003, 3)

In [7]:
def simplistic_mesh_creator_voxel(pointCloudMatrix):
     #takes a numpy matrix of size [x,3] of points 
     #and uses open3d inorder to create a basic voxel representation of the points
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
        
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
   # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    o3d.visualization.draw_geometries([vox_mesh]) #visualization
    
    #still need to export properly
        
    

In [8]:
simplistic_mesh_creator_voxel(pointCloudMatrix)

In [9]:


import matplotlib.pyplot as plt

In [10]:
def simplistic_mesh_creator_poisson(pointCloudMatrix):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
    #o3d.visualization.draw_geometries([pcd])

    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3))) #remove exiting Normals
    #pcd.estimate_normals() #Estimate new normals base on pointcloud
    radius = 0.01   # 
    max_nn = 30     # 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
    #pcd.orient_normals_consistent_tangent_plane(k=15)
    #pcd.orient_normals_consistent_tangent_plane(100) #Crashes program
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=9)
    o3d.visualization.draw_geometries([mesh])
    print('visualize densities')
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    o3d.visualization.draw_geometries([density_mesh])
    print('remove low density vertices')
    vertices_to_remove = densities < np.quantile(densities, 0.01)
    mesh.remove_vertices_by_mask(vertices_to_remove)
    print(mesh)
    o3d.visualization.draw_geometries([mesh])


In [11]:
simplistic_mesh_creator_poisson(pointCloudMatrix)

[Open3D DEBUG] Input Points / Samples: 60003 / 60002
[Open3D DEBUG] #   Got kernel density: 0.186 (s), 474.305 (MB) / 474.305 (MB) / 474 (MB)
[Open3D DEBUG] #     Got normal field: 0.113 (s), 484.281 (MB) / 484.281 (MB) / 484 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 1.408486e-04 / 8.451340e+00
[Open3D DEBUG] #       Finalized tree: 0.261 (s), 489.711 (MB) / 489.711 (MB) / 505 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.521 (s), 481.238 (MB) / 489.711 (MB) / 505 (MB)
[Open3D DEBUG] #Set point constraints: 0.129 (s), 475.516 (MB) / 489.711 (MB) / 505 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1287588 / 511496 / 960033
[Open3D DEBUG] Memory Usage: 475.527 MB
[Open3D DEBUG] # Linear system solved: 1.079 (s), 508.738 (MB) / 508.738 (MB) / 508 (MB)
[Open3D DEBUG] Got average: 0.0350001 (s), 475.496 (MB) / 508.738 (MB) / 508 (MB)
[Open3D DEBUG] Iso-Value: 4.994412e-01 = 2.996797e+04 / 6.000300e+04
[Open3D DEBUG] #          Total Solve:       4.1 (s),     508.7 (M